# MaDDash and central esmond Measurement Archive
<p>The following steps will install the `perfsonar-centralmanagement` bundle to provide a standalone MaDDash server and central esmond Measurement Archive (MA)</p>

<p>At the end of the session we will have a central MA to which to register GridFTP and perfSONAR test results and dashboards to visualize performance:</p>
<ul>
<li>initial system setup, base installation, and initial configuration</li>
<li>build and publish a mesh json for consumption by the MaDDash server and participating perfSONAR nodes</li>
<li>update the MaDDash configuration for GridFTP</li>
<li>establish credentials for nodes to use in registering GridFTP and perfSONAR results</li>
</ul>

## pull in the perfsonar-centralmanagement notebook
<p>using your terminal app log into your fionette as root, and cd into /root</p>
<p>use wget to transfer the perfsonar-centralmanagement notebook<pre>
wget http://fiona-10-01:8000/perfsonar-centralmanagement.ipynb -O perfsonar-centralmanagement.ipynb
</pre>
<p>launch jupyter lab</p>
<pre>jupyter lab --allow-root --ip=`your.fionette.ip`</pre>

## initial system setup

In [ ]:
uname -r

In [ ]:
sed -i 's/SELINUX=enforcing/SELINUX=disabled/g' /etc/selinux/config
setenforce 0

## install NTP, bind-utils, traceroute if not already present

In [ ]:
yum -y install ntp bind-utils traceroute

In [ ]:
systemctl enable ntpd

# adjust `/etc/ntp.conf` for the workshop NTP server
<p>review the contents of the file</p>

In [ ]:
cat /etc/ntp.conf

<p>using your favorite editor, comment out NTP servers outside of the workshop and add an entry for the local NTP server</p>
<pre>
#server time.apple.com
server 137.164.48.196 iburst
</pre>

## verify the changes

In [ ]:
cat /etc/ntp.conf

In [ ]:
systemctl start ntpd

In [ ]:
systemctl status ntpd -l

In [ ]:
ntpq -p -c rv

## determine default interface

In [ ]:
default_iface=$(awk '$2 == 00000000 { print $1 }' /proc/net/route)
echo $default_iface

## confirm `hostname -f` responds with FQHN

In [ ]:
hostname -f

## use hostnamectl to set if its not a valid FQHN

In [ ]:
myip=$(ip addr show dev $default_iface | grep 'inet '| awk '{print $2}' | awk -F "/" '{print $1}')
dig -x $myip +short

In [ ]:
hostnamectl set-hostname `dig -x $myip +short`
hostname -f

## view current link settings on default interface

In [ ]:
ip link show dev $default_iface

## create a custom udev rules file for tuning the network configuration 

In [ ]:
if [ ! -f /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules ]; then
cat > /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules <<EOL
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k mtu 9000"
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k txqueuelen 10000"
EOL
chmod 644 /etc/udev/rules.d/80-prp-fiona-ws-ma-netconf.rules
fi

## Use udevadm to reload the rules and trigger the action

In [ ]:
/sbin/udevadm control --reload-rules
/sbin/udevadm trigger

## confirm desired settings are in effect

In [ ]:
ip link show dev $default_iface

## Create a custom sysctl file with desired settings

In [ ]:
if [ ! -f /etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf ]; then
cat>/etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf <<EOL
net.core.rmem_max = 67108864 
net.core.wmem_max = 67108864 
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control=htcp
EOL
chmod 644 /etc/sysctl.d/80-prp-fiona-ws-ma-sysctl.conf
fi

## Adjust kernel boot-time defaults and rebuild grub.cfg

In [ ]:
if ! grep --quiet default_hugepagesz /etc/default/grub; then
	sed -i 's/^GRUB_CMDLINE_LINUX=\"[^\"]*/& default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce/' /etc/default/grub
	grub2-mkconfig -o /boot/grub2/grub.cfg
fi

In [ ]:
cat /etc/default/grub

# SAVE your work NOW

In [ ]:
reboot

## confirm intended tuning settings are present 

In [ ]:
ip link show | grep UP | grep -v lo

## disable a few unwanted services and enable the `tuned` service

In [ ]:
systemctl stop NetworkManager 
systemctl disable NetworkManager
systemctl stop irqbalance.service
systemctl disable irqbalance.service
systemctl stop wpa_supplicant
systemctl disable wpa_supplicant
systemctl start tuned
systemctl enable tuned


## check the current clock rate of the CPU cores

In [ ]:
cat /proc/cpuinfo | grep Hz

## use `tuned-adm` to show the current active profile

In [ ]:
tuned-adm active

## set `throughput-performance` at the active profile

In [ ]:
tuned-adm profile throughput-performance

## re-check the clock rate of the CPU cores

In [ ]:
cat /proc/cpuinfo | grep Hz

# Installing the perfSONAR Central Management bundle
<p>The instructions below follow the perfSONAR CentOS Bundle Installation page, here:</p>
<p>http://docs.perfsonar.net/install_centos.html</p>

## point yum to local perfSONAR repo 

In [ ]:
cat << EOF > /etc/yum.repos.d/perfsonar.repo
[perfsonar]
name=Extra Packages for Enterprise Linux 7 - $basearch
baseurl=http://137.164.48.196/centos-7-x86_64/perfsonar
failovermethod=priority
enabled=1
gpgcheck=0
EOF

## Refresh the yum cache 

In [ ]:
yum clean all

## Now we are ready to install the `perfsonar-centralmanagement` bundle:
<p>due to the large number of packages involved, the call to yum in the next cell includes the `-q` switch to set yum to quiet mode.  output to `stderr` and any non-zero exit code will appear in the cell.</p>
<p>NOTE: launch a separate terminal session and run <pre>tail -f /var/log/yum.log</pre> to follow the installation process.</p>

In [ ]:
yum -y -q install perfsonar-centralmanagement

## use `systemctl` to enable services

In [ ]:
systemctl enable httpd
systemctl start httpd
systemctl enable cassandra
systemctl start cassandra

## check firewall rules

In [ ]:
firewall-cmd --list-all

## configure the firewall rules to allow http / https

In [ ]:
firewall-cmd --permanent --zone=public --add-service=http
firewall-cmd --permanent --zone=public --add-service=https
firewall-cmd --reload

## verify updates to the firewall ruleset

In [ ]:
firewall-cmd --list-all

## SAVE your work NOW
## Reboot the server 


In [ ]:
reboot

## verify things came up as expected

In [ ]:
systemctl status maddash-server -l
systemctl status perfsonar-meshconfig-guiagent -l
systemctl status postgresql-9.5 -l
systemctl status cassandra -l
systemctl status httpd -l

## verify the MaDDash webUI is available
<p>the following cell will make a call to `curl` to request the text of the MaDDash webUI main page.</p>
<p>alternatively, launch a new web browser tab to view the dashboard.</p>

In [ ]:
curl http://$(hostname -s)/maddash-webui/

## verify the esmond MA is operational
<p>without any data in the MA at this point the expected output is an open and close pair of square brackets</p>

In [ ]:
curl http://$(hostname -s)/esmond/perfsonar/archive/?format=json

## create an adminstrative user for the MaDDash webUI
<p>MaDDash provides an adminstrative interface available through the webUI at <pre>http://enter.the.ma.ip.here/maddash-webui/admin</pre> <p>in a termal session, use `htpasswd` to update `/etc/maddash/maddash-webui/admin-users`</p>
<pre>htpasswd /etc/maddash/maddash-webui/admin-users admin</pre>

# Creating and publishing a perfSONAR mesh json
<p>the next set of steps will produce and publish an updated mesh json:
<ul>
<li>process the mesh json to update MaDDash server yaml configuration: configuring checks to query the esmond; thresholds for performance, and organizing dashboards</li>
<li>publish the mesh json for participating perfSONAR nodes: which process the mesh json to update their test configuration</li>
</ul>
</p>

## use wget to transfer a baseline mesh configuration (.conf)
<p>the mesh .conf file provides a more human-readable configuration syntax (than json).</p>
<p>subsequent steps reference the mesh .conf to build the mesh json</p>

In [ ]:
cd /var/www/html
wget http://fiona-10-01:8000/prp-fiona-ws-mesh--$(hostname -s).conf -O prp-fiona-ws-mesh--$(hostname -s).conf

## check the contents of the baseline mesh .conf
<p>use your favorite editor to make any necessary changes</p>

In [ ]:
cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf

## generate the perfSONAR mesh json

In [ ]:
/usr/lib/perfsonar/bin/build_json --verbose /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf > /var/www/html/prp-fiona-ws-mesh--$(hostname -s).json

## view the mesh json

In [ ]:
cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).json

## validate the generated perfSONAR mesh json

In [ ]:
/usr/lib/perfsonar/bin/validate_json --verbose --config_url=http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json

# Update the MaDDash server yaml configuration
<p>in the next series of steps we will:</p>
<ul>
<li>set up a fresh `maddash.yaml` configuration on which to subsequently apply changes</li>
<li>update the `meshconfig-guiagent.conf` file to create a mesh referencing the newly created perfSONAR mesh json</li>
<li>update the `maddash.yaml` for GridFTP checks, thresholds, and dashboards</li>
</ul>

## create a fresh `maddash.yaml` file

In [ ]:
if [ -f /etc/maddash/maddash-server/maddash.yaml ]; then
mv /etc/maddash/maddash-server/maddash.yaml /etc/maddash/maddash-server/maddash.yaml.orig
cat>/etc/maddash/maddash-server/maddash.yaml <<EOL
##
# Set the directory where the database will be stored
database: /var/lib/maddash/

##
# Set the number of jobs that can run in parallel. Default is 20.
#jobThreadPoolSize: 20

##
# Set number of jobs that can be in queue at one time. Default is 250.
#jobBatchSize: 250

###
# Disable the job scheduler if you only want to run the REST server
#disableScheduler: 0

###
# Skips table and index rebuild at start-up. It can speed up start-up time if set to 1.
#skipTableBuild: 0

##
# Set the host where the REST server listens
serverHost: "localhost"

##
# Activate http and set the port where it listens
http:
    port: 8881
EOL
chown maddash:maddash /etc/maddash/maddash-server/maddash.yaml
chmod 644 /etc/maddash/maddash-server/maddash.yaml
fi

## view the `maddash.yaml` file

In [ ]:
cat /etc/maddash/maddash-server/maddash.yaml

## update `meshconfig-guiagent.conf` to add a `<mesh>` stanza referencing the mesh json</p>

In [ ]:
if [ ! -f /etc/perfsonar/meshconfig-guiagent.conf.orig ]; then
cp /etc/perfsonar/meshconfig-guiagent.conf /etc/perfsonar/meshconfig-guiagent.conf.orig
cat >> /etc/perfsonar/meshconfig-guiagent.conf <<EOL
## You can define more meshes to configure against by adding more 'mesh' blocks.
<mesh>
    configuration_url             http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json
#    #validate_certificate         0
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
</mesh>
EOL
elif [ -f /etc/perfsonar/meshconfig-guiagent.conf.orig ]; then
rm -f /etc/perfsonar/meshconfig-guiagent.conf
cp /etc/perfsonar/meshconfig-guiagent.conf.orig /etc/perfsonar/meshconfig-guiagent.conf
cat >> /etc/perfsonar/meshconfig-guiagent.conf <<EOL
## You can define more meshes to configure against by adding more 'mesh' blocks.
<mesh>
    configuration_url             http://$(hostname -s)/prp-fiona-ws-mesh--$(hostname -s).json
#    #validate_certificate         0
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
</mesh>
EOL
fi

## review the updated `meshconfig-guiagent.conf`

In [ ]:
cat /etc/perfsonar/meshconfig-guiagent.conf 

## restart the `perfsonar-meshconfig-guiagent` process

In [ ]:
systemctl restart perfsonar-meshconfig-guiagent

## review the updated `maddash.yaml`

In [ ]:
cat /etc/maddash/maddash-server/maddash.yaml

## restart the `maddash-server` process


In [ ]:
systemctl restart maddash-server

## browse to the MaDDAsh server webUI to confirm the updates

# Update MaDDash server configuration file for visualizing GridFTP disk-to-disk throughput  


## apply a patch file to a copy of `maddash.yaml` to insert sections for GridFTP-related checks, dashboards and grids, and groups

In [ ]:
cd /etc/maddash/maddash-server/
cat maddash.yaml > maddash--$(hostname -s)--ps.yaml
wget http://fiona-10-01:8000/maddash--$(hostname -s)--ps-gridftp.yaml.patch -O maddash--$(hostname -s)--ps-gridftp.yaml.patch
patch --input=maddash--$(hostname -s)--ps-gridftp.yaml.patch --output=maddash--$(hostname -s)--ps-gridftp.yaml maddash--$(hostname -s)--ps.yaml

## sanity check the diff of the copy of `maddash.yaml` and patched file

In [ ]:
diff -U 4 maddash--$(hostname -s)--ps.yaml maddash--$(hostname -s)--ps-gridftp.yaml

## replace `maddash.yaml` with the patched file

In [ ]:
cat maddash--$(hostname -s)--ps-gridftp.yaml > maddash.yaml

# update the MaDDash webUI `config.json` to reflect the customized dashboard naming
<p>review the current contents of <pre>/etc/maddash/maddash-webui/config.json</pre></p>

In [ ]:
cd /etc/maddash/maddash-webui
cat config.json

## use `wget` to transfer an customized `config.json` and replace the original

In [ ]:
wget http://fiona-10-01:8000/webui--$(hostname -s)--config.json -O webui--$(hostname -s)--config.json
cat webui--$(hostname -s)--config.json > config.json

## restart the `maddash-server` process 
<p>NOTE: in a separate terminal session, run <pre>tail -f /var/log/maddash/maddash-server.netlogger.log</pre> to check for errors</p>

In [ ]:
systemctl restart maddash-server

## browse to the MaDDash webUI to confirm the new GridFTP dashboards are present
<p>NOTE: you will need to clear your browsers cache in order to view the results of the updated webUI config.json</p>
<p>ex: with Chrome, select <pre>Clear Browsing Data...</pre> to bring up a dialog of options.  clearing the last hour of browsing data should be sufficient</p>

# Update esmond to add credentials to register GridFTP transfer results
<p>the following steps will update your central esmond MA to authtenticate GridFTP nodes by an API key</p>  
<p>on the nodes running GridFTP transfers, we will use the generated API key to configure the script which parses the transfer logs and registers the throughput results to the esmond MA</p>
<p>NOTE: you may re-run this cell and it will display the previously generated API key</p>


In [ ]:
cd /usr/lib/esmond
. bin/activate
python esmond/manage.py add_api_key_user gridftp

# Update esmond to add credentials for pS nodes to register results
<p>the following cell will update your central esmond MA to authtenticate pS nodes to by their IP address</p>

In [ ]:
cd /usr/lib/esmond
. bin/activate
for node in $(cat /var/www/html/prp-fiona-ws-mesh--$(hostname -s).conf | grep member | awk '{ print $2 }'); do  python esmond/manage.py add_user_ip_address $node `dig +short $node` ; done


# Update nodes to schedule GridFTP transfers and perfSONAR tests, and register results to the central MA 
<p>in the next session, we will work on the nodes running GridFTP server and perfSONAR testpoint to update their configurations</p>
<p>this would be a good time to SAVE your WORK in this notebook</p>